In [1]:
pip install selenium beautifulsoup4


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


import libraries

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
from collections import Counter
import time

set driver path

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
#from webdriver_manager.chrome import ChromeDriverManager

options = Options()
#options.add_argument("--headless")  # Run headless, if desired
#service=Service(ChromeDriverManager().install()) # Run in the backgroun
driver_path = r'C:\Users\Sejal\Downloads\nlp_project\chromedriver-win64\chromedriver.exe'
options = webdriver.ChromeOptions()
# Initialize ChromeDriver
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=options)

initialize

In [4]:
# Initialize ChromeDriver
#service = Service(driver_path)

function to search on diff search engines

In [7]:
# List of search engines and their URLs
search_engines = {
    'Google': 'https://www.google.com',
    'Yahoo': 'https://search.yahoo.com',
    'Bing': 'https://www.bing.com',
    'Brave': 'https://search.brave.com',
    'DuckDuckGo': 'https://www.duckduckgo.com'  # Added DuckDuckGo
}

def get_search_results(engine_name, search_url, search_query):
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(search_url)

    try:
        # DuckDuckGo-specific scraping logic
        if engine_name == 'DuckDuckGo':
            print("Opening DuckDuckGo...")
            search_box = driver.find_element(By.NAME, 'q')
            search_box.send_keys(search_query)
            search_box.send_keys(Keys.RETURN)
            print("Search query submitted on DuckDuckGo.")

            # Wait for results to load
            time.sleep(3)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            print("Page parsed with BeautifulSoup for DuckDuckGo.")

            # Extract links from DuckDuckGo results
            results = soup.find_all('a', {'data-testid': 'result-title-a'})  # Attribute is more stable
            urls = {result['href'] for result in results[:5]}  # Limit to top 5 results
            return urls

        # Brave-specific scraping logic
        elif engine_name == 'Brave':
            print("Opening Brave search engine...")
            search_box = driver.find_element(By.NAME, 'q')
            search_box.send_keys(search_query)
            search_box.send_keys(Keys.RETURN)
            time.sleep(2)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            snippets = soup.find_all('div', class_='snippet svelte-umc06')
            urls = {snippet.find('a')['href'] for snippet in snippets if snippet.find('a')}
            return urls

        # Other search engines (Google, Yahoo, Bing)
        else:
            if engine_name == 'Google':
                search_box = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.NAME, 'q')))
            elif engine_name == 'Yahoo':
                search_box = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.NAME, 'p')))
            elif engine_name == 'Bing':
                search_box = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.NAME, 'q')))
            else:
                print(f"Search box not found on {engine_name}")
                driver.quit()
                return set()

            search_box.send_keys(search_query)
            search_box.send_keys(Keys.RETURN)
            time.sleep(2)
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            urls = set()
            if engine_name == 'Google':
                results = soup.find_all('div', class_='g')
                urls = {result.find('a')['href'] for result in results[:5] if result.find('a')}
            elif engine_name == 'Yahoo':
                results = soup.find_all('h3', class_='title')
                urls = {result.find('a')['href'] for result in results[:5] if result.find('a')}
            elif engine_name == 'Bing':
                results = soup.find_all('li', class_='b_algo')
                urls = {result.find('a')['href'] for result in results[:5] if result.find('a')}

            return urls

    except Exception as e:
        print(f"Error scraping {engine_name}: {e}")
        return set()

    finally:
        driver.quit()

function to extract url

In [8]:
# Function to extract text from a URL
def extract_text(url):
    try:
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text = ' '.join(paragraph.text for paragraph in paragraphs)
        return text[:1000]  # Limit to first 1000 characters
    except Exception as e:
        print(f"Error extracting text from {url}: {e}")
        return ""

main function

In [ ]:
# Main function
def main():
    search_query = "AI trends latest by 28/Oct"
    all_results = {}

    # Get URLs from each search engine
    for engine_name, search_url in search_engines.items():
        print(f"Searching on {engine_name}...")
        urls = get_search_results(engine_name, search_url, search_query)
        all_results[engine_name] = urls
        print(f"Top URLs from {engine_name}:")
        
        # Loop through all scraped URLs for this search engine
        for url in urls:
            print(url)
            # Extract text for each URL
            extracted_text = extract_text(url)
            print("Extracted text:", extracted_text)
            print("-" * 50)

    # Optional: Find common links across search engines (if needed for further processing)
    all_urls = [url for urls in all_results.values() for url in urls]
    url_counts = Counter(all_urls)
    common_links = [url for url, count in url_counts.items() if count > 1]

    # Select the top 3 common links
    top_common_links = common_links[:3]
    print("\nTop 3 common links:")
    for link in top_common_links:
        print(link)
        print("Extracted text:", extract_text(link))
        print("-" * 50)

    # Select 2 unique (non-common) links from each search engine
    for engine_name, urls in all_results.items():
        unique_links = [url for url in urls if url not in common_links]
        top_unique_links = unique_links[:2]
        
        print(f"\nTop 2 unique links from {engine_name}:")
        for link in top_unique_links:
            print(link)
            print("Extracted text:", extract_text(link))
            print("-" * 50)

if __name__ == "__main__":
    main()



Searching on Google...
Top URLs from Google:
https://builtin.com/artificial-intelligence/artificial-intelligence-future#:~:text=What%20does%20the%20future%20of,and%20worries%20over%20job%20losses.
Extracted text: 
                    AI is constantly changing our world. Here are just a few ways AI will influence our lives.
                 Innovations in the field of artificial intelligence continue to shape the future of humanity across nearly every industry. AI is already the main driver of emerging technologies like big data, robotics and IoT, and generative AI has further expanded the possibilities and popularity of AI.  According to a 2023 IBM survey, 42 percent of enterprise-scale businesses integrated AI into their operations, and 40 percent are considering AI for their organizations. In addition, 38 percent of organizations have implemented generative AI into their workflows while 42 percent are considering doing so. With so many changes coming at such a rapid pace, here’s what